In [1]:
import sys

import numpy as np
import osmnx as ox
import pandas as pd

ox.__version__

'2.0.3'

In [17]:
!gdal_translate \
  -a_srs EPSG:31370 \
  DTM_20m.tif \
  DTM_20m_l72.tif

zsh:1: command not found: gdal_translate


In [ ]:
!gdalwarp \
  -s_srs EPSG:31370 \
  -t_srs EPSG:32631 \
  -r bilinear \
  DTM_20m_l72.tif \
  DTM_20m_utm31.tif

In [2]:
import osmnx as ox

# geocode Leuven and get its boundary polygon as a GeoDataFrame
gdf = ox.geocode_to_gdf("Leuven, Belgium")  
# extract [minx, miny, maxx, maxy] = [west, south, east, north]
west, south, east, north = gdf.geometry.total_bounds

print("Bbox:", west, south, east, north)

Bbox: 4.640295 50.8242096 4.7705305 50.9440707


In [3]:
!osmium extract \
  --strategy complete_ways \
  --bbox 4.640295,50.8242096,4.7705305,50.9440707 \
  belgium-latest.osm.pbf \
  -o G.osm.pbf

[======================================================================] 100% 


In [5]:
from pyrosm import OSM

osm = OSM("G.osm.pbf")
gdf_nodes, gdf_edges = osm.get_network(network_type="driving", nodes=True)

In [9]:
G = osm.to_graph(
    nodes=gdf_nodes,
    edges=gdf_edges,
    graph_type="networkx",
    osmnx_compatible=True
)

In [14]:
# Add node elevations from raster files (replace with your Belgium DEM files)
belgium_raster_path = "DTM_20m.tif"
try:
    G = ox.elevation.add_node_elevations_raster(G, belgium_raster_path, cpus=1)
except Exception as e:
    print(f"Could not load Belgium raster file: {e}")

In [15]:
# Add edge grades and their absolute values
G = ox.elevation.add_edge_grades(G, add_absolute=True)

In [16]:
# Get a color for each edge by grade, then plot the network

# Remove edges with missing or invalid 'grade_abs' before plotting
import numpy as np

edges_with_grade = [
    (u, v, k)
    for u, v, k, d in G.edges(keys=True, data=True)
    if d.get("grade_abs") is not None and not np.isnan(d.get("grade_abs"))
]
if len(edges_with_grade) == 0:
    print("No valid 'grade_abs' values found on edges. Cannot plot.")
else:
    G_valid = G.edge_subgraph(edges_with_grade).copy()
    try:
        ec_belgium = ox.plot.get_edge_colors_by_attr(
            G_valid, "grade_abs", cmap="plasma", num_bins=5, equal_size=True
        )
    except ValueError as e:
        print(f"Warning: {e}\nFalling back to equal_size=False.")
        try:
            ec_belgium = ox.plot.get_edge_colors_by_attr(
                G_valid, "grade_abs", cmap="plasma", num_bins=5, equal_size=False
            )
        except ValueError as e2:
            print(f"Plotting failed: {e2}")
            ec_belgium = None
    if ec_belgium is not None:
        fig, ax = ox.plot.plot_graph(
            G_valid, edge_color=ec_belgium, edge_linewidth=0.5, node_size=0, bgcolor="k"
        )

No valid 'grade_abs' values found on edges. Cannot plot.
